# Ternary Diagrams for Cluster Search from IVAS

In this jupyter notebook, you will be able to:
- quickly load your cluster search file,
- filter the elements,
- plot the clusters on:
    - a stacked bar chart,
    - a ternary diagram,
    - XYZ map with locations and size of each cluster

Requirements:
Plotly is not installed on anaconda or python3.6 on default.
To install it, run ```$ pip install plotly``` on your Linux terminal, or ```$ conda install plotly``` on your anaconda terminal.

In [1]:
import numpy as np
import pandas 
import plotly.graph_objects as go
import plotly.express as px

In [2]:
### YOUR INPUT NEEDED ###
# delete first rows in your IVAS csv file - I can't find the way to put it in the code yet
# remember to change \ to /
cluster_data_file_path = "C:/Users/Peter/Downloads/8829-v01SiNiMnPCuDpt52N17.csv"
### THANKS ###

In [3]:
# run this if on WSL, skip if you're on "normal" Windows
cluster_data_file_path = cluster_data_file_path.replace("C:", "/mnt/c")
print(cluster_data_file_path)

/mnt/c/Users/Peter/Downloads/8829-v01SiNiMnPCuDpt52N17.csv


In [4]:
# checking if you stated the correct file
df = pandas.read_csv(cluster_data_file_path, sep=",", error_bad_lines=False)
df.head()

,Unnamed: 0,Solute Ions,Ranged Ions,Total Ions,Unnamed: 4,Center_x (nm) Solute,Center_y (nm) Solute,Center_z (nm) Solute,R_gx (nm) Solute,R_gy (nm) Solute,...,Si Count,Ni Count,Cr Count,Mn Count,P Count,Cu Count,O Count,MoN Count,MoO Count,N Count
0,Matrix,64076,2547690,2795347,NaN,NaN,NaN,NaN,NaN,NaN,...,10373,25268,2997,27175,555,705,1624,374,266,0
1,Cluster 1,20,35,36,NaN,1.8940,-1.7282,0.5568,0.4294,0.4221,...,7,10,0,3,0,0,0,0,0,0
2,Cluster 2,59,107,109,NaN,4.7156,-1.0052,1.6256,0.8790,0.6332,...,9,17,0,31,2,0,0,0,0,0
3,Cluster 3,17,34,35,NaN,-1.9973,4.6410,1.1301,0.2229,0.2952,...,3,7,0,7,0,0,0,0,0,0
4,Cluster 4,25,56,57,NaN,-3.1440,-6.8237,1.9649,0.6268,0.3839,...,3,6,0,15,1,0,0,0,0,0


In [5]:
### YOUR INPUT NEEDED ###
# which ions do you want to plot?
core_ions = ["Si", "Mn", "Ni", "Cu", "P"]
plot_with_Fe = True # change to False if needed
n_min = 5
n_max = 999999
### THANKS ###

# I exclude Fe from cluster size
exclude_ions = ["Fe Count"]

core_ion_colors = {
    "Cu": "orange",
    "Ni": "green",
    "Mn": "yellow",
    "P": "pink",
    "Si": "grey"
}

# change the name of the column
df["Cluster ID"] = df.iloc[:,0]

In [8]:
# create a new figure
fig = go.Figure()

# open the cluster stats file and create pandas dataframe
# remove matrix from the cluster search
local_df = df[df["Cluster ID"] != "Matrix"]

# add total ions in cluster value = Ranged Ions - Fe ions
local_df["Cluster Size"] = local_df["Ranged Ions"] - local_df["Fe Count"]

# change n_min
local_df = local_df[local_df["Cluster Size"] >= n_min]
local_df = local_df[local_df["Cluster Size"] <= n_max]

# sort it by size
local_df = local_df.sort_values(by="Cluster Size")

### YOUR FILTERS ###
# uncomment and adjust if needed

# # if you want to plot only clusters with, e.g. Ni content greater than 40%
# # create a column with floats
# local_df["my filter values"] = local_df["Ni Count"] / local_df["Cluster Size"] * 100
# # create / modify local_df with positive indexes 
# local_df = local_df[local_df["my filter values"] >= 40]



# iterate through rows and find their core ions and total ions values
x_values = np.arange(len(local_df["Cluster Size"]))
for i2, core_ion in enumerate(core_ions):
    core_ion_IVAS = core_ion + " Count"
    y_values = local_df[core_ion_IVAS] / local_df["Cluster Size"] * 100
    
#         # for each cluster size find all clusters and calculate mean core ion content with stdev
#         for i3, x in enumerate(iter(x_values)):
#             all_for_one_size = local_df[local_df["Cluster Size"] == x][core_ion]
#             y_values[i3] = all_for_one_size.mean() / x * 100
#             y_std[i3] = all_for_one_size.std() / x * 100
#             # TODO: think about how to plot st dev later on

    fig.add_trace(go.Bar(
        x=x_values, 
        y=y_values, 
        name=core_ion,
        marker_color=core_ion_colors[core_ion]
    ))

  
    
# reopen pandas dataframe to get the Fe content and plot a line
if plot_with_Fe:
    Fe_concentration = local_df["Fe Count"] / local_df["Ranged Ions"] * 100
    fig.add_trace(go.Line(x=x_values, y=Fe_concentration, name="Fe", opacity=0.7, line_color="red"))
    
# adjust the plot
fig.update_layout(
    barmode="stack", 
    title_text="Core Ion concentration"
)

# Set custom x-axis labels
how_many_ticks = 15
print(len(x_values))
interval = round(len(x_values) / how_many_ticks)


ticktext_data = local_df["Cluster Size"][::interval]
ticktvals_data = x_values[::interval]


fig.update_xaxes(
    ticktext=ticktext_data,
    tickvals=ticktvals_data,
    title_text="Cluster Size in Ions"
)

fig.update_yaxes(
    title_text="Concentration in at. %"
)

fig.show()


# prepare columns for XYZ plotting 
local_df["X Axis"] = local_df["Center_x (nm) Solute"]
local_df["Y Axis"] = local_df["Center_y (nm) Solute"]
local_df["Z Axis"] = local_df["Center_z (nm) Solute"] * -1

# prepare columns for ternary diagrams
local_df["Si"] = local_df["Si Count"]
local_df["Mn"] = local_df["Mn Count"]
local_df["Ni"] = local_df["Ni Count"]


# plotting using plotly ternary diagrams
#     fig = px.scatter_ternary(local_df, a="relative_Ni_Mn", b="relative_Si", c="relative_Cu")
fig2 = px.scatter_ternary(
    local_df, a="Si", b="Ni", c="Mn", 
    hover_name="Cluster ID", # change later to show cluster ID
    size="Cluster Size", size_max=10,
    color="Z Axis", color_continuous_scale=px.colors.sequential.Viridis,
    opacity=0.6
#         marker=dict(
#             color='LightSkyBlue',
#             size='Relative Cluster Size',
#             size_max = 15,
#             opacity=0.5,
#             line=dict(
#                 color='MediumPurple',
#                 width=5
#             )
#         )
)
fig2.show()

# plot the clusters in XYZ and cluster size
fig3 = px.scatter_3d(
    local_df, x='X Axis', y='Y Axis', z='Z Axis',
    color='Z Axis', color_continuous_scale=px.colors.sequential.Viridis,
    size='Cluster Size', size_max=15,
    opacity=0.6,
    hover_name="Cluster ID"
)

fig3.show()


292


In [23]:
# plot mean compositions on ternary diagrams
fig = go.Figure()


# ion = "Cu"
x_axis_values = np.arange(20, 160, 20) # ONE STEP LONGER THAN YOU WANT IT TO BE
ions_to_plot = ["Mn", "Ni", "Si"] # ONLY THREE
if len(ions_to_plot) != 3:
    print("You have to specify exactly three elements / columns!")

all_ternary_dfs = []

# development only 
# delete later
sample_titles = ["Sample 1"]
dfs = [local_df]
    

for i, sample in enumerate(sample_titles):
    
    # find bin stats to smooth the graphs and generalise the data
    
    # local_cs_normalised = local_cs / etas[i] # cluster size / LEAP efficiency
    local_rgyration = dfs[i]["Cluster Size"] / 0.52

    # create new columns if needed
    dfs[i]["Mn+Si"] = dfs[i]["Si"] + dfs[i]["Mn"]

    # iterate through every bin and find mean and std of ion
    y_axis_values = [0, 0, 0]
    y_axis_values[0] = np.zeros(len(x_axis_values)-1)
    y_axis_values[1] = np.zeros(len(x_axis_values)-1)
    y_axis_values[2] = np.zeros(len(x_axis_values)-1)

    y_axis_error = [0, 0, 0]
    y_axis_error[0] = np.zeros(len(x_axis_values)-1)
    y_axis_error[1] = np.zeros(len(x_axis_values)-1)
    y_axis_error[2] = np.zeros(len(x_axis_values)-1)

    bin_cluster_count = np.zeros(len(x_axis_values)-1)

    for i2, bin in enumerate(list(x_axis_values[1:])):
        not_lower_than = local_rgyration > x_axis_values[i2]
        not_higher_than = local_rgyration < x_axis_values[i2+1]
        mask = not_higher_than & not_lower_than
       
        # iterate through three ions to plot
        for i3, ion in enumerate(ions_to_plot):
            
            bin_ion_mean = np.mean(dfs[i][ion][mask])
            bin_ion_std = np.std(dfs[i][ion][mask])
            try:
                bin_error = bin_ion_std / (len(dfs[i][ion][mask])**0.5) # SEM
            except ZeroDivisionError:
                bin_error = 0
            y_axis_values[i3][i2] = bin_ion_mean
            y_axis_error[i3][i2] = bin_error

        # find cluster count in each bin
        bin_cluster_count[i2] = len(dfs[i][ion][mask])


    # move all the data to pandas df, it's easier to plot this way
    this_df = pandas.DataFrame()
    
    this_df["Cluster Radius"] = x_axis_values[:-1]
    
    this_df[ions_to_plot[0]] = y_axis_values[0]
    this_df[ions_to_plot[1]] = y_axis_values[1]
    this_df[ions_to_plot[2]] = y_axis_values[2]

    this_df["Bin Cluster Count"] = bin_cluster_count
    column_name = "{} Error"
    this_df[column_name.format(ions_to_plot[0])] = y_axis_error[0]
    this_df[column_name.format(ions_to_plot[1])] = y_axis_error[1]
    this_df[column_name.format(ions_to_plot[2])] = y_axis_error[2]

    # add sample column
    this_df["Sample"] = np.array(([sample] * len(bin_cluster_count))) 

    # add informative cluster size ranges for hover id
    ticktext = []
    tickvals = []
    for i2 in range(len(x_axis_values[1:])):
        text = "{:.1f}-{:.1f}".format(x_axis_values[i2], x_axis_values[i2+1])
        ticktext.append(text)
        tickvals.append(x_axis_values[i2])
    this_df["Radius Range [nm]"] = ticktext

    # add to the list of all dataframes and concatenate later
    # all_ternary_dfs.append(this_df)

    # for development only
    fig.add_trace(go.Scatterternary(
        a=this_df[ions_to_plot[0]], 
        b=this_df[ions_to_plot[1]], 
        c=this_df[ions_to_plot[2]], 
        opacity=0.7,
        mode="lines+markers",
        name=sample,
        marker={
            # 'symbol': 100,
            'color': 'green',
            'size': [8, 9, 10, 11, 12, 13, 14]
            }
    ))

def makeAxis(title, tickangle):
    return {
      'title': title,
      'titlefont': { 'size': 15 },
      'tickangle': tickangle,
      'tickfont': { 'size': 15 },
      'tickcolor': 'rgba(0,0,0,0)',
      'ticklen': 5,
      'showline': True,
      'showgrid': True
    }

fig.update_layout({
    'ternary': {
        # 'sum': 100,
        'aaxis': makeAxis(ions_to_plot[0], 0),
        'baxis': makeAxis(ions_to_plot[1], 45),
        'caxis': makeAxis(ions_to_plot[2], -45)
    },
    'annotations': [{
      'showarrow': False,
      'text': 'How concentration changes with size',
        'x': 0.5,
        'y': 1.3,
        'font': { 'size': 15 }
    }]
})


fig.show()

In [24]:
this_df

,Cluster Radius,Mn,Ni,Si,Bin Cluster Count,Mn Error,Ni Error,Si Error,Sample,Radius Range [nm]
0,20,8.054054,6.414414,3.441441,111.0,0.208121,0.179036,0.176395,Sample 1,20.0-40.0
1,40,10.256410,9.230769,4.606838,117.0,0.268695,0.237288,0.193758,Sample 1,40.0-60.0
2,60,14.444444,14.066667,6.333333,45.0,0.492467,0.495311,0.298142,Sample 1,60.0-80.0
3,80,18.083333,17.083333,8.500000,12.0,0.923898,0.803277,0.506897,Sample 1,80.0-100.0
4,100,24.666667,22.333333,10.666667,3.0,2.841492,2.227771,0.720082,Sample 1,100.0-120.0
5,120,27.500000,27.750000,9.750000,4.0,1.600781,2.558686,1.138804,Sample 1,120.0-140.0
